In [48]:
import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fastf1.core import Session
from fastf1.api import SessionNotAvailableError
import requests

# Configure matplotlib for inline plotting
%matplotlib inline

In [49]:
fastf1.Cache.enable_cache('../data')  # Cache data in the 'data' directory
fastf1.Cache.offline_mode(True) # Enable offline mode to prevent network traffic

In [50]:
# Gets session like race or qualifying

def collect_session_data(year, race_round, session_name):
    try:
        session = fastf1.get_session(year, race_round, session_name)
        session.load()
        if not session.laps.empty:  # Check if laps were loaded successfully
            return session
        else:
            print(f"Laps data not available for {year} round {race_round} in {session_name} session.")
            return None
    except SessionNotAvailableError:
        print(f"Session not available for {year} round {race_round}")
        return None

In [51]:
# Cell 4: Extracting Lap and Telemetry Data with Enhanced Error Handling

def extract_lap_telemetry_data(session):
    laps = session.laps
    lap_data = laps[['Driver', 'LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
                     'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
                     'Compound', 'TyreLife', 'Team']]
    telemetry_data = pd.DataFrame()

    # Process lap-by-lap telemetry for each driver
    for driver in session.drivers:
        try:
            driver_laps = laps.pick_driver(driver)
            if not driver_laps.empty:
                fastest_lap = driver_laps.pick_fastest()
                if session.car_data is not None and driver in session.car_data:
                    car_data = fastest_lap.get_car_data()  # Retrieve telemetry data
                    car_data['Driver'] = driver  # Tag driver in telemetry data
                    telemetry_data = pd.concat([telemetry_data, car_data])
                else:
                    print(f"Car data unavailable for driver {driver}")
        except KeyError:
            print(f"Missing telemetry data for driver {driver}")
        except Exception as e:
            print(f"Error processing driver {driver}: {e}")

    return lap_data, telemetry_data


In [52]:
# Cell 5: Collecting Event and Driver Metadata

def collect_event_metadata(session):
    event_info = {}

    # Check if session.event is available before accessing attributes
    if session.event is not None:
        event_info = {
            'event_name': session.event.get('EventName', 'N/A'),
            'country': session.event.get('Country', 'N/A'),
            'location': session.event.get('Location', 'N/A'),
            'event_date': session.event.get('EventDate', 'N/A'),
            'race_round': session.event.get('RoundNumber', 'N/A')
        }
    else:
        print("Event data is unavailable for this session.")

    # Attempt to get circuit information only if pos_data is available
    if session.pos_data is not None:
        try:
            circuit_info = session.get_circuit_info()
            if circuit_info:
                event_info['corner_count'] = len(circuit_info.corners)
                event_info['rotation'] = circuit_info.rotation  # Circuit rotation in degrees
        except (AttributeError, KeyError):
            print("Circuit information is unavailable or could not be loaded due to missing telemetry data.")
    else:
        print("Position data is unavailable; skipping circuit information.")

    # Collect driver information if session.drivers is available
    driver_info_list = []
    for driver in session.drivers:
        driver_data = session.get_driver(driver)
        driver_info_list.append({
            'Driver': driver,
            'Team': getattr(driver_data, 'team', 'N/A'),
            'DriverNumber': getattr(driver_data, 'number', 'N/A')
        })
    driver_info = pd.DataFrame(driver_info_list)

    return event_info, driver_info


In [55]:
# Cell 6: Aggregating Data for Multiple Races and Sessions with Race-Only Filtering

def collect_all_race_data(start_year, end_year, sessions=['Q', 'R']):
    all_data = []
    for year in range(start_year, end_year + 1):
        try:
            # Get the event schedule for the current year and filter for race events only
            schedule = fastf1.get_event_schedule(year)
            race_schedule = schedule[schedule['EventFormat'] != 'testing']  # Exclude test events
            rounds = race_schedule['RoundNumber'].unique()  # Get list of valid race rounds
        except ValueError:
            print(f"Failed to retrieve schedule for {year}. Skipping year.")
            continue

        for round_number in rounds:
            for session_name in sessions:
                session = collect_session_data(year, round_number, session_name)
                if session:
                    lap_data, telemetry_data = extract_lap_telemetry_data(session)
                    event_info, driver_info = collect_event_metadata(session)

                    # Combine all data into a dictionary
                    data_entry = {
                        'event_info': event_info,
                        'driver_info': driver_info,
                        'lap_data': lap_data,
                        'telemetry_data': telemetry_data
                    }
                    all_data.append(data_entry)

                    print(f"Data collected for {year} round {round_number} - {session_name}")

    return all_data


In [56]:
collected_data = collect_all_race_data(2018, 2024)

core           INFO 	Loading data for Australian Grand Prix - Qualifying [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '7', '5', '33', '3', '20', '8', '27', '55', '77', '14', '2', '11', '18', '31', '28', '9', '16', '35', '10']
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNIN

Data collected for 2018 round 1 - Q


req            INFO 	Using cached data for car_data
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
core        WARNING 	Car position data is unavailable!
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '44', '7', '3', '14', '33', '27', '77', '2', '55', '11', '31', '16', '18', '28', '8', '20', '10', '9', '35']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timi

Circuit information is unavailable or could not be loaded due to missing telemetry data.
Data collected for 2018 round 1 - R


req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '7', '77', '44', '3', '10', '20', '27', '31', '55', '28', '11', '14', '2', '33', '8', '9', '35', '16', '18']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver 27: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver  2: Lap timing integrity check failed for 1 lap(s)
core        WAR

...

Data collected for 2024 round 20 - Q


core        WARNING 	Cannot load lap times for first lap from Ergast. Timing data is not available for this session.
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['55', '1', '4', '16', '63', '44', '20', '10', '23', '27', '22', '30', '14', '18', '77', '43', '81', '11', '24', '31']
/Users/karrtopelka/Developer/lnu/f1-v1/venv/lib/python3.12/site-packages/fastf1/core.py:3131: FutureWarning: In the future, `None` will be returned instead of an empty `Lap` object when there are no laps that satisfy the definition for fastest lap.
  warnings.warn(("In the future, `None` will be returned instead of "
/Users/karrtopelka/Developer/lnu/f1-v1/venv/lib/python3.12/site-packages/fastf1/core.py:3131: FutureWarning: In the future, `None` will be returned instead 

Missing telemetry data for driver 23
Missing telemetry data for driver 22
Data collected for 2024 round 20 - R


logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNING 	Failed to load track status data!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
logger 

DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`

In [47]:
collected_data[0]['event_info']

NameError: name 'collected_data' is not defined